In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import sentencepiece as spm  # <--- THAY ĐỔI QUAN TRỌNG
import math
import time
import os
from tqdm import tqdm

# --- CẤU HÌNH ĐƯỜNG DẪN (SỬA LẠI CHỖ NÀY) ---
# Thư mục chứa các file: train.bpe.en, train.bpe.vi, iwslt_bpe.model
INPUT_DIR = "/kaggle/input/iwslt-dataprocessing" 
BPE_MODEL_PATH = '/kaggle/input/tokenizer-iwslt/iwslt_bpe.model'

# Các file dữ liệu (Đã tokenize BPE ở bước trước)
TRAIN_SRC_FILE = '/kaggle/input/tokenizer-iwslt/train.bpe.en'
TRAIN_TRG_FILE = '/kaggle/input/tokenizer-iwslt/train.bpe.vi'
# (Tạm thời dùng train làm valid nếu bạn chưa tạo file valid.bpe riêng)
VAL_SRC_FILE =  '/kaggle/input/tokenizer-iwslt/valid.bpe.en'
VAL_TRG_FILE = '/kaggle/input/tokenizer-iwslt/valid.bpe.vi'

# --- HYPERPARAMETERS ---
MAX_LEN = 256      # Giảm xuống chút cho nhẹ nếu cần
BATCH_SIZE = 128
N_EPOCHS = 40
LEARNING_RATE = 0.0005
CLIP = 1

# Load SentencePiece để lấy thông số Vocab
sp = spm.SentencePieceProcessor()
sp.load(BPE_MODEL_PATH)

INPUT_DIM = sp.get_piece_size() # Lấy tự động từ file model (khoảng 16000)
OUTPUT_DIM = sp.get_piece_size()
D_MODEL = 256
N_HEAD = 4
D_FF = 1024
N_LAYERS = 4
DROP_PROB = 0.3
PATIENCE = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Vocab Size loaded: {INPUT_DIM}")
print(f"Device: {device}")

Vocab Size loaded: 16000
Device: cuda


In [2]:
class TranslationDataset(Dataset):
    def __init__(self, src_path, trg_path, sp_model_path):
        # Load SentencePiece Model
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(sp_model_path)
        
        # Lấy ID của các token đặc biệt
        self.bos_id = self.sp.bos_id()
        self.eos_id = self.sp.eos_id()
        self.pad_id = self.sp.pad_id()
        
        print(f"⏳ Đang đọc dữ liệu từ: {os.path.basename(src_path)} & {os.path.basename(trg_path)}...")
        with open(src_path, 'r', encoding='utf-8') as f:
            self.src_data = f.readlines()
        with open(trg_path, 'r', encoding='utf-8') as f:
            self.trg_data = f.readlines()
            
    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        # 1. Lấy dòng text BPE (VD: "_hello _world")
        src_line = self.src_data[idx].strip()
        trg_line = self.trg_data[idx].strip()
        
        # 2. Chuyển thành List tokens (String) -> List IDs (Int)
        # Lưu ý: Dùng piece_to_id vì text đã được tokenize sẵn
        src_ids = self.sp.piece_to_id(src_line.split())
        trg_ids = self.sp.piece_to_id(trg_line.split())
        
        # 3. Thêm BOS và EOS vào đầu cuối
        src_tensor = torch.tensor([self.bos_id] + src_ids + [self.eos_id])
        trg_tensor = torch.tensor([self.bos_id] + trg_ids + [self.eos_id])
        
        return src_tensor, trg_tensor

In [3]:
# --- HÀM COLLATE (GOM BATCH) ---
class Collate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    def __call__(self, batch):
        src, trg = zip(*batch)
        # Pad để các câu trong batch bằng nhau
        src = pad_sequence(src, padding_value=self.pad_idx, batch_first=True)
        trg = pad_sequence(trg, padding_value=self.pad_idx, batch_first=True)
        return src, trg

In [4]:
# --- KHỞI TẠO DATASET & DATALOADER ---
# Lấy PAD ID từ model đã load ở cell 1
PAD_IDX = sp.pad_id() 

print("🔄 Đang khởi tạo Train Loader...")
train_ds = TranslationDataset(TRAIN_SRC_FILE, TRAIN_TRG_FILE, BPE_MODEL_PATH)
train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=Collate(PAD_IDX), 
    num_workers=2
)

print("🔄 Đang khởi tạo Valid Loader...")
# Lưu ý: Nếu bạn chưa có file valid riêng, dùng tạm file train (hoặc split ra)
# Ở đây mình demo dùng luôn file config ở cell 1
val_ds = TranslationDataset(VAL_SRC_FILE, VAL_TRG_FILE, BPE_MODEL_PATH)
valid_loader = DataLoader(
    val_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=Collate(PAD_IDX), 
    num_workers=2
)

print(f"✅ Đã sẵn sàng! Train size: {len(train_ds)}, Valid size: {len(val_ds)}")

🔄 Đang khởi tạo Train Loader...
⏳ Đang đọc dữ liệu từ: train.bpe.en & train.bpe.vi...
🔄 Đang khởi tạo Valid Loader...
⏳ Đang đọc dữ liệu từ: valid.bpe.en & valid.bpe.vi...
✅ Đã sẵn sàng! Train size: 128068, Valid size: 1553


In [5]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, trg in iterator:
            src, trg = src.to(device), trg.to(device)
            trg_input, trg_label = trg[:, :-1], trg[:, 1:]
            
            output = model(src, trg_input)
            output = output.contiguous().view(-1, output.shape[-1])
            trg_label = trg_label.contiguous().view(-1)
            
            loss = criterion(output, trg_label)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [6]:
class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len, drop_prob):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        emb = self.tok_emb(x) * math.sqrt(self.d_model)
        pos = self.pe[:, :x.size(1)]
        return self.dropout(emb + pos)

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super().__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.head_dim = d_model // n_head
        
        # Đảm bảo d_model chia hết cho số head
        assert self.head_dim * n_head == d_model, "d_model phải chia hết cho n_head"

        # 1. Các lớp Linear để chiếu Q, K, V
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        
        # Lớp Linear cuối cùng sau khi nối các head lại
        self.w_o = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        """
        q, k, v shape: [Batch_Size, Seq_Len, d_model]
        mask shape: [Batch_Size, 1, 1, Seq_Len] hoặc [Batch_Size, 1, Seq_Len, Seq_Len]
        """
        batch_size = q.size(0)

        # 1. Chiếu Q, K, V qua Linear layer
        # Sau đó tách thành n_head: [Batch, Seq, Head, Dim] -> [Batch, Head, Seq, Dim]
        # Transpose để đưa chiều Head lên trước chiều Seq -> Để nhân ma trận song song các head
        Q = self.w_q(q).view(batch_size, -1, self.n_head, self.head_dim).transpose(1, 2)
        K = self.w_k(k).view(batch_size, -1, self.n_head, self.head_dim).transpose(1, 2)
        V = self.w_v(v).view(batch_size, -1, self.n_head, self.head_dim).transpose(1, 2)

        # 2. Tính Scaled Dot-Product Attention
        # Score = (Q * K^T) / sqrt(d_k)
        # K.transpose(-2, -1) là chuyển vị 2 chiều cuối (Seq, Dim) -> (Dim, Seq)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # 3. Áp dụng Mask (Nếu có)
        # Mask thường chứa 0 (che) và 1 (giữ). Ta thay vị trí 0 bằng số âm vô cùng (-1e9)
        # để khi qua Softmax nó biến thành 0.
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # 4. Softmax để ra xác suất
        attention_weights = torch.softmax(scores, dim=-1)
        
        # 5. Nhân với V
        # Output: [Batch, Head, Seq, Dim]
        output = torch.matmul(attention_weights, V)
        
        # 6. Gom các head lại (Concatenate)
        # [Batch, Head, Seq, Dim] -> [Batch, Seq, Head, Dim] -> [Batch, Seq, d_model]
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # 7. Đi qua lớp Linear cuối cùng
        return self.w_o(output)

In [8]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, drop_prob=0.1):
        super().__init__()
        # d_ff thường lớn gấp 4 lần d_model (ví dụ: 512 -> 2048)
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        # x: [Batch, Seq_Len, d_model]
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [9]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff, drop_prob=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(drop_prob)
        
        self.ffn = PositionwiseFeedForward(d_model, d_ff, drop_prob)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(drop_prob)

    def forward(self, x, mask=None):
        # 1. Sub-layer 1: Self Attention
        # Lưu lại x ban đầu để cộng (Residual Connection)
        _x = x
        x = self.attention(q=x, k=x, v=x, mask=mask) # Self-Attention: q=k=v=x
        x = self.dropout1(x)
        x = self.norm1(x + _x) # Add & Norm
        
        # 2. Sub-layer 2: Feed Forward
        _x = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + _x) # Add & Norm
        
        return x

In [10]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, d_ff, n_layer, max_len, drop_prob, device):
        super().__init__()
        self.device = device
        
        # Embedding + Positional Encoding (Đã code ở bài trước)
        self.embedding = TransformerEmbedding(vocab_size, d_model, max_len, drop_prob)
        
        # Chồng N lớp EncoderLayer
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_head, d_ff, drop_prob) 
            for _ in range(n_layer)
        ])
        
    def forward(self, src, mask=None):
        # src: [Batch, Seq_Len]
        x = self.embedding(src)
        
        # Cho đi qua lần lượt từng lớp Encoder
        for layer in self.layers:
            x = layer(x, mask)
        
        return x

In [11]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff, drop_prob=0.1):
        super().__init__()
        
        # 1. Self Attention (Có Mask che tương lai)
        self.self_attention = MultiHeadAttention(d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(drop_prob)
        
        # 2. Cross Attention (Quan trọng: Lấy Key, Value từ Encoder)
        self.cross_attention = MultiHeadAttention(d_model, n_head)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(drop_prob)
        
        # 3. Feed Forward
        self.ffn = PositionwiseFeedForward(d_model, d_ff, drop_prob)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout3 = nn.Dropout(drop_prob)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        """
        trg: Input của Decoder (câu tiếng Việt đang dịch dở)
        enc_src: Output từ Encoder (câu tiếng Anh đã hiểu xong)
        trg_mask: Mask che tương lai cho trg
        src_mask: Mask che padding cho src
        """
        # --- Block 1: Masked Self-Attention ---
        # Decoder tự nhìn lại chính nó (nhưng không được nhìn tương lai)
        _trg = trg
        # Quan trọng: trg_mask dùng ở đây
        trg = self.self_attention(q=trg, k=trg, v=trg, mask=trg_mask)
        trg = self.dropout1(trg)
        trg = self.norm1(trg + _trg) # Add & Norm

        # --- Block 2: Cross-Attention (Encoder-Decoder Attention) ---
        # Decoder lấy thông tin từ Encoder
        # Query (Q) đến từ Decoder (trg)
        # Key (K) và Value (V) đến từ Encoder (enc_src)
        _trg = trg
        # Quan trọng: src_mask dùng ở đây (để không nhìn vào padding của tiếng Anh)
        trg = self.cross_attention(q=trg, k=enc_src, v=enc_src, mask=src_mask)
        trg = self.dropout2(trg)
        trg = self.norm2(trg + _trg)

        # --- Block 3: Feed Forward ---
        _trg = trg
        trg = self.ffn(trg)
        trg = self.dropout3(trg)
        trg = self.norm3(trg + _trg)

        return trg

In [12]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, d_ff, n_layer, max_len, drop_prob, device):
        super().__init__()
        self.device = device
        
        # Embedding riêng cho Decoder (Tiếng Việt)
        self.embedding = TransformerEmbedding(vocab_size, d_model, max_len, drop_prob)
        
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_head, d_ff, drop_prob)
            for _ in range(n_layer)
        ])
        
        # Lớp Linear cuối cùng để dự đoán từ tiếp theo
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        # trg: [Batch, Seq_Len]
        trg = self.embedding(trg)
        
        for layer in self.layers:
            trg = layer(trg, enc_src, trg_mask, src_mask)
            
        # Output: [Batch, Seq_Len, Vocab_Size]
        output = self.fc_out(trg)
        return output

In [13]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        # src shape: [Batch, Src_Len]
        
        # Tạo mask cho vị trí padding (True nếu != pad, False nếu == pad)
        # Hoặc ngược lại tùy quy ước, ở đây ta dùng quy ước: 1 là giữ, 0 là che
        # unsqueeze(1) và (2) để mở rộng chiều cho khớp với Attention Heads
        # Shape mong muốn: [Batch, 1, 1, Src_Len]
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        # trg shape: [Batch, Trg_Len]
        
        # 1. Padding Mask: Che các vị trí pad trong câu đích
        # Shape: [Batch, 1, 1, Trg_Len]
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        # 2. Look-ahead Mask: Ma trận tam giác
        trg_len = trg.shape[1]
        # torch.tril tạo ma trận tam giác dưới (số 1 ở dưới đường chéo, số 0 ở trên)
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=self.device)).bool()
        
        # 3. Kết hợp cả 2: Vừa phải không phải pad, vừa phải nằm trong tam giác dưới
        # Shape: [Batch, 1, Trg_Len, Trg_Len]
        trg_mask = trg_pad_mask & trg_sub_mask
        
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        """
        src: [Batch, Src_Len]
        trg: [Batch, Trg_Len] (Lưu ý: trg này là Input cho Decoder, đã bỏ token cuối)
        """
        # 1. Tạo Mask
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        # 2. Chạy qua Encoder
        enc_src = self.encoder(src, src_mask)
        
        # 3. Chạy qua Decoder
        # Lưu ý: Decoder cần cả src_mask để tránh Cross-Attention nhìn vào padding của src
        output = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        return output

In [14]:
def initialize_weights(m):
    """
    Hàm khởi tạo trọng số Xavier (Glorot) Uniform.
    Rất quan trọng để Transformer hội tụ nhanh khi train từ đầu.
    """
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

print("🚀 Khởi tạo Model MỚI (Train from scratch)...")

# 1. Khởi tạo các thành phần
enc = Encoder(INPUT_DIM, D_MODEL, N_HEAD, D_FF, N_LAYERS, MAX_LEN, DROP_PROB, device)
dec = Decoder(OUTPUT_DIM, D_MODEL, N_HEAD, D_FF, N_LAYERS, MAX_LEN, DROP_PROB, device)

# 2. Tạo Model tổng
# Lưu ý: PAD_IDX lấy từ biến sp.pad_id() ở cell trên
model = Transformer(enc, dec, PAD_IDX, PAD_IDX, device).to(device)

# 3. Áp dụng khởi tạo trọng số (QUAN TRỌNG)
# model.apply(initialize_weights)
# print("✨ Đã khởi tạo tham số ngẫu nhiên (Xavier Init).")

# 4. Kiểm tra số lượng tham số
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"📊 Tổng số tham số (Trainable Parameters): {count_parameters(model):,}")

🚀 Khởi tạo Model MỚI (Train from scratch)...
📊 Tổng số tham số (Trainable Parameters): 19,676,800


In [15]:
# # --- 2. LOAD TOKENIZER & LẤY VOCAB SIZE ---
# # Phải có bước này để biết INPUT_DIM/OUTPUT_DIM là bao nhiêu
# BPE_MODEL_PATH = os.path.join(INPUT_DIR, "/kaggle/input/tokenizer-iwslt/iwslt_bpe.model")
# sp = spm.SentencePieceProcessor()
# sp.load(BPE_MODEL_PATH)

# VOCAB_SIZE = sp.get_piece_size()
# PAD_IDX = sp.pad_id()

# # 👇 DÒNG BẠN HỎI LÀ Ở ĐÂY. BẮT BUỘC PHẢI CÓ!
# print("🏗️ Đang xây dựng kiến trúc Model...")
# enc = Encoder(VOCAB_SIZE, D_MODEL, N_HEAD, D_FF, N_LAYERS, MAX_LEN, DROP_PROB, device)
# dec = Decoder(VOCAB_SIZE, D_MODEL, N_HEAD, D_FF, N_LAYERS, MAX_LEN, DROP_PROB, device)
# model = Transformer(enc, dec, PAD_IDX, PAD_IDX, device).to(device)

In [16]:
import math
import time

# --- CẤU HÌNH TRAIN TIẾP (PHASE 2) ---
PRETRAINED_PATH = '/kaggle/input/transformer-training-en2vi/transformer_best_en2vi_finetunedV2.pt'  # File gốc (Epoch 40)
NEW_SAVE_PATH = 'transformer_small_en2vi_v2.pt'      # File mới (Epoch 50)
EXTRA_EPOCHS = 10

# 1. Load trọng số cũ
print(f"🔄 Loading weights from {PRETRAINED_PATH}...")
model.load_state_dict(torch.load(PRETRAINED_PATH, map_location=device))

# 2. Optimizer LR thấp & cố định (cho Fine-tuning)
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.0001,  # LR nhỏ để nhích từ từ
    betas=(0.9, 0.98), 
    eps=1e-9,
    weight_decay=1e-4
)

criterion = nn.CrossEntropyLoss(
    ignore_index=PAD_IDX, 
    label_smoothing=0.1
)

# 3. Reset Best Loss (Lấy mốc của model cũ, ví dụ 3.4 hoặc 3.5)
best_valid_loss = 3.5 

print(f"🚀 Bắt đầu Train thêm {EXTRA_EPOCHS} epochs (No Progress Bar)...")
print(f"{'Epoch':^5} | {'Train Loss':^10} | {'Val Loss':^10} | {'Val PPL':^10} | {'Time':^10}")
print("-" * 55)

for epoch in range(EXTRA_EPOCHS):
    start_time = time.time()
    
    # --- TRAIN ---
    model.train()
    train_loss = 0
    
    # Loop trực tiếp qua loader, không dùng tqdm
    for src, trg in train_loader:
        src, trg = src.to(device), trg.to(device)
        trg_input, trg_label = trg[:, :-1], trg[:, 1:]
        
        optimizer.zero_grad()
        output = model(src, trg_input)
        output = output.contiguous().view(-1, output.shape[-1])
        trg_label = trg_label.contiguous().view(-1)
        
        loss = criterion(output, trg_label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        optimizer.step()
        
        train_loss += loss.item()

    # --- EVALUATE ---
    # (Đảm bảo bạn có hàm evaluate trả về loss trung bình)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    # Tính toán chỉ số
    avg_train_loss = train_loss / len(train_loader)
    valid_ppl = math.exp(valid_loss) if valid_loss < 100 else float('inf')
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    # --- IN KẾT QUẢ ---
    print(f"{epoch+1:^5} | {avg_train_loss:^10.3f} | {valid_loss:^10.3f} | {valid_ppl:^10.3f} | {int(epoch_mins)}m {int(epoch_secs)}s")
    
    # --- SAVE ---
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), NEW_SAVE_PATH)
        print(f"      --> 💾 Saved Best V2 (Loss: {valid_loss:.3f})")

print("✅ DONE FINE-TUNING!")

🔄 Loading weights from /kaggle/input/transformer-training-en2vi/transformer_best_en2vi_finetunedV2.pt...
🚀 Bắt đầu Train thêm 10 epochs (No Progress Bar)...
Epoch | Train Loss |  Val Loss  |  Val PPL   |    Time   
-------------------------------------------------------
  1   |   3.215    |   3.405    |   30.115   | 3m 53s
      --> 💾 Saved Best V2 (Loss: 3.405)
  2   |   3.189    |   3.400    |   29.951   | 3m 52s
      --> 💾 Saved Best V2 (Loss: 3.400)
  3   |   3.178    |   3.395    |   29.810   | 3m 52s
      --> 💾 Saved Best V2 (Loss: 3.395)
  4   |   3.173    |   3.390    |   29.662   | 3m 52s
      --> 💾 Saved Best V2 (Loss: 3.390)
  5   |   3.170    |   3.399    |   29.937   | 3m 53s
  6   |   3.168    |   3.397    |   29.879   | 3m 52s
  7   |   3.168    |   3.395    |   29.827   | 3m 52s
  8   |   3.167    |   3.392    |   29.719   | 3m 51s
  9   |   3.167    |   3.393    |   29.752   | 3m 52s
 10   |   3.168    |   3.396    |   29.857   | 3m 53s
✅ DONE FINE-TUNING!
